**12-04 영어/한국어 Word2Vec 학습시키기**

In [1]:
# 영어 Word2Vec 만들기

import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")


('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7b60aff370d0>)

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
#훈련데이터 전처리하기
targetXML = open('ted_en-20160408.xml', 'r', encoding = 'UTF8')
target_text = etree.parse(targetXML)

#xml 파일로부터 <content> 와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

#정규 표현식의 sub 모듈을 통해 배경음 부분을 제거
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)','',parse_text)

#입력 코퍼스에 대해 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 소문자화
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [5]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [6]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


In [7]:
# Word2Vec 훈련시키기
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size = 100, window = 5, min_count =5, workers=4, sg=0)

In [8]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8414031863212585), ('guy', 0.7985521554946899), ('boy', 0.7604570984840393), ('lady', 0.747848629951477), ('gentleman', 0.738959014415741), ('soldier', 0.7295765280723572), ('girl', 0.7167888283729553), ('rabbi', 0.6883883476257324), ('kid', 0.6761054396629333), ('poet', 0.6708167195320129)]


In [9]:
#모델 저장
model.wv.save_word2vec_format('eng_w2v')
loaded_model = KeyedVectors.load_word2vec_format('eng_w2v')

In [ ]:
pip install konlpy

In [13]:
#한국어 Word2Vec 만들기

import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [14]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7b6041e6cbb0>)

In [15]:
train_data = pd.read_table('ratings.txt')
train_data[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [16]:
print(len(train_data))

200000


In [17]:
print(train_data.isnull().values.any())

True


In [18]:
train_data = train_data.dropna(how='any')
print(train_data.isnull().values.any())

False


In [19]:
print(len(train_data))

199992


In [20]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document']=train_data['document'].str.replace('[^ㄱ-하-ㅣ가-힣]',"")

<ipython-input-20-3925aa735c93>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document']=train_data['document'].str.replace('[^ㄱ-하-ㅣ가-힣]',"")


In [21]:
train_data[:5]

,id,document,label
0,8112052,어릴때보고지금다시봐도재밌어요ㅋㅋ,1
1,8132799,디자인을배우는학생으로외국디자이너와그들이일군전통을통해발전해가는문화산업이부러웠는데사실우...,1
2,4655635,폴리스스토리시리즈는부터뉴까지버릴께하나도없음최고,1
3,9251303,와연기가진짜개쩔구나지루할거라고생각했는데몰입해서봤다그래이런게진짜영화지,1
4,10067386,안개자욱한밤하늘에떠있는초승달같은영화,1


In [25]:
from tqdm import tqdm

#불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

#형태소 분석기 OKT를 사용한 토큰화 작업
okt = Okt()

tokenized_data = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    tokenized_data.append(stopwords_removed_sentence)

  5%|▍         | 9465/199992 [05:07<1:43:07, 30.79it/s]


KeyboardInterrupt: 

In [ ]:
print('리뷰의 최대 길이 :', max(len(review) for review in tokenized_data))
print('리뷰의 평균 길이 :', sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(review) for review in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

**12-05 임베딩 벡터의 시각화**

In [26]:
!python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng_w2v

2024-02-29 11:54:49,714 - word2vec2tensor - INFO - running /usr/local/lib/python3.10/dist-packages/gensim/scripts/word2vec2tensor.py --input eng_w2v --output eng_w2v
2024-02-29 11:54:49,714 - keyedvectors - INFO - loading projection weights from eng_w2v
2024-02-29 11:54:50,914 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (21613, 100) matrix of type float32 from eng_w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2024-02-29T11:54:50.913321', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.1.58+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}
2024-02-29 11:54:52,232 - word2vec2tensor - INFO - 2D tensor file saved to eng_w2v_tensor.tsv
2024-02-29 11:54:52,232 - word2vec2tensor - INFO - Tensor metadata file saved to eng_w2v_metadata.tsv
2024-02-29 11:54:52,233 - word2vec2tensor - INFO - finished running word2vec2tensor.py


In [27]:
model.wv.most_similar('electrofishing')

KeyError: "Key 'electrofishing' not present in vocabulary"

In [31]:
from gensim.models import FastText

model = FastText(result,vector_size=100, window=5, min_count=5, workers = 4, sg=1)

In [32]:
model.wv.most_similar('electrofishing')

[('electrolyte', 0.8643126487731934),
 ('electrolux', 0.8608806729316711),
 ('electroencephalogram', 0.8535165786743164),
 ('electro', 0.8477595448493958),
 ('electroshock', 0.8475503325462341),
 ('electrochemical', 0.8268005847930908),
 ('electrogram', 0.8257236480712891),
 ('electrons', 0.8167737722396851),
 ('electron', 0.8145217299461365),
 ('electrode', 0.8140037655830383)]